In [ ]:
print('hello')

### Manifest

In [ ]:
# ! python3 /home/saurabh/transliterated/translate_copy.py \
#     --root_dir=/home/saurabh/dataset/openslr/kanada/labelled \
#     --dst_folder=/home/saurabh/manifest/new/kanada_labelled \
#     --valid_percent=0.2 

### TOkenizer

In [ ]:
# #wpe - working on
# ! python3 /home/saurabh/nemo/NeMo/scripts/tokenizers/process_asr_text_tokenizer.py \
#     --data_file=/home/saurabh/dataset/corpus_data_for_tokenizer/dakshini/kn/wiki-filt.train.text.shuf.txt \
#     --data_root=/home/saurabh/manifest/new/kanada_labelled/wpe_50 \
#     --vocab_size=50 \
#     --tokenizer=wpe

In [ ]:
# # Bpe 
# ! python3 /home/saurabh/nemo/NeMo/scripts/tokenizers/process_asr_text_tokenizer.py \
#     --manifest=/home/saurabh/manifest/new/kanada_labelled/train.json \
#     --data_root=/home/saurabh/manifest/new/kanada_labelled/spe_1000 \
#     --vocab_size=1000 \
#     --tokenizer=spe

In [ ]:
# ## Tokenizer using large corpus
# ! python3 /home/saurabh/nemo/NeMo/scripts/tokenizers/process_asr_text_tokenizer.py \
#     --data_file=/home/saurabh/dataset/corpus_data_for_tokenizer/kanada/kn.txt \
#     --data_root=/home/saurabh/manifest/new/kanada_labelled/spe_from_corpus \
#     --vocab_size=32000 \
#     --tokenizer=spe \
#     --spe_train_extremely_large_corpus
    

In [ ]:
# # Bpe 
# ! python3 /home/saurabh/nemo/NeMo/scripts/tokenizers/process_asr_text_tokenizer.py \
#     --manifest=/home/saurabh/manifest/new/kanada_labelled/train.json \
#     --data_root=/home/saurabh/manifest/new/kanada_labelled/spe \
#     --vocab_size=100 \
#     --tokenizer=spe

# # #wpe    
# ! python3 /home/saurabh/nemo/NeMo/scripts/tokenizers/process_asr_text_tokenizer.py \
#     --manifest=/home/saurabh/manifest/new/kanada_labelled/train.json \
#     --data_root=/home/saurabh/manifest/new/kanada_labelled/wpe \
#     --vocab_size=100 \
#     --tokenizer=wpe

### Model From Scratch

In [ ]:
#### Import Dependencies ####
import nemo.collections.asr as nemo_asr
from nemo.utils import exp_manager
import pytorch_lightning
from omegaconf import OmegaConf, open_dict
from nemo.core import adapter_mixins
from pytorch_lightning import Trainer
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig
import torch

from pytorch_lightning.loggers import WandbLogger

# Char
config_path = '/home/saurabh/nemo/NeMo/examples/asr/conf/conformer/conformer_ctc_char.yaml'
params = OmegaConf.load(config_path)
params.model.labels = ["೨", "ೇ", "ಛ", "ಿ", "ಞ", "ಊ", "೧", "ಾ", "n", "'", "ಆ", "೯", "ಉ", "ಂ", "ಢ", "೪", "೮", "ಲ", "ೌ", "ಙ", "ವ", "ಬ", "ಧ", "ಗ", "ೋ", "ಳ", "ಥ", "ಭ", "ಚ", "೩", "ಖ", "’", "ಓ", "ಠ", "ಈ", "ಕ", "ಔ", "ು", "ಋ", "ತ", "ಐ", "ೀ", "ೆ", "ಮ", "ಘ", "೫", "ೃ", "ೈ", "೬", "ಷ", "ಏ", "ದ", "ರ", "ಇ", "್", "೭", "ಫ", "ನ","ೊ", "ೂ", "ಅ", "ಪ", "ಝ", "ಜ", "ಒ", "ಯ", "ಶ", "ಣ", "ಟ", "ಎ", "ಸ", "ಹ", "ಃ", "ಡ"]


# params.model.tokenizer.dir = '/home/saurabh/manifest/new/kanada_labelled/spe_1000/tokenizer_spe_bpe_v1000'
# params.model.tokenizer.type = "bpe"

print(OmegaConf.to_yaml(params))

accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
# accelerator = 'cpu'
max_steps = -1
max_epochs = 100

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=3, 
                  max_epochs=max_epochs)


train_manifest = '/home/saurabh/manifest/new/kanada_labelled/train.json'
test_manifest  = '/home/saurabh/manifest/new/kanada_labelled/valid.json' 

# Update paths to dataset
params.model.train_ds.manifest_filepath = train_manifest
params.model.validation_ds.manifest_filepath = test_manifest

# remove spec augment for this dataset
params.model.spec_augment.rect_masks = 0

# d_model | n_heads | n_layers | time_masks | lr  |
#   176   |    4    |    16    |     5      | 5.0 |

# COnformer CTC SMall
params.model.spec_augment.time_masks = 10
params.model.encoder.d_model = 176
params.model.encoder.n_layers = 16
params.model.encoder.n_heads = 4
# params.optim.lr = 5

# OmegaConf.set_struct(params, True)
with open_dict(params):
    params.model.tokenizer.dir = '/home/saurabh/manifest/new/kanada_labelled'
    params.model.tokenizer.type = 'wpe'
    
exp_direc = '/home/saurabh/experiments/kanada_conformer_wpe'
name_exp = 'kanada'
#wandb Kargs
name_wandb = 'run01'
project_wandb = 'kanada'

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=exp_direc,
    name=name_exp,
    
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor='val_wer',
        mode='min',
        # always_save_nemo=True,
        save_best_model=False,
    ),
    
    resume_if_exists=True,
    resume_ignore_no_checkpoint=True,
    create_tensorboard_logger= False,
    create_checkpoint_callback= True,
  
    create_wandb_logger=True,
    wandb_logger_kwargs={
        'name':name_wandb, 
        'project':project_wandb
    },
)

exp_config=OmegaConf.structured(exp_config)
logdir = exp_manager.exp_manager(trainer, exp_config)

In [ ]:
params.model.optim.lr = 5

In [ ]:
first_asr_model = nemo_asr.models.EncDecCTCModelBPE(cfg=params.model, trainer=trainer)

In [ ]:
print(OmegaConf.to_yaml(first_asr_model.cfg))

In [ ]:
trainer.fit(first_asr_model)

In [ ]:
# ## COnformer CTC

# import time
# start_time = time.time()

# !python /home/saurabh/python/nemo/NeMo/examples/asr/asr_ctc/speech_to_text_ctc.py \
# --config-path=/home/saurabh/python/nemo/NeMo/examples/asr/conf/conformer \
# --config-name=conformer_ctc_char \
# model.train_ds.manifest_filepath=/home/saurabh/python/notebooks/nemo_new_18feb/manifest/assamese/train.json \
# model.validation_ds.manifest_filepath=/home/saurabh/python/notebooks/nemo_new_18feb/manifest/assamese/valid.json\
# trainer.max_epochs=4 \
# model.labels=$labels_hindi \
# model.encoder.n_layers=8 \
# model.encoder.n_heads=2

# end_time = time.time()
# print(f'\n*** Time: {end_time-start_time} secs***')

In [ ]:
## Conformer RNNT

### Save Model , load for finetuning

### Finetuning